In [ ]:
!pip install flask pyngrok diffusers torch 

In [ ]:
pip install peft

In [ ]:
import threading
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif
from flask import Flask, request, send_file
import os

class AnimateLCMModel:
    def __init__(self):
        self.pipe = None
        self.setup_model()

    def setup_model(self):
        adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)
        self.pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
        self.pipe.scheduler = LCMScheduler.from_config(self.pipe.scheduler.config, beta_schedule="linear")
        self.pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
        self.pipe.set_adapters(["lcm-lora"], [0.8])
        self.pipe.enable_vae_slicing()
        self.pipe.enable_model_cpu_offload()

    def generate_video(self, prompt, negative_prompt, num_frames, guidance_scale, num_inference_steps, seed):
        generator = torch.Generator("cpu").manual_seed(int(seed))
        output = self.pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_frames=num_frames,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            generator=generator,
        )
        frames = output.frames[0]
        return frames

model = AnimateLCMModel()

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data['prompt']
    negative_prompt = data['negative_prompt']
    num_frames = data['num_frames']
    guidance_scale = data['guidance_scale']
    num_inference_steps = data['num_inference_steps']
    seed = data['seed']
    frames = model.generate_video(prompt, negative_prompt, num_frames, guidance_scale, num_inference_steps, seed)
    output_path = "output.gif"
    export_to_gif(frames, output_path)
    return send_file(output_path, mimetype='image/gif')

def run_flask():
    app.run(host='0.0.0.0', port=5000)

# Start the Flask server in a separate thread
thread = threading.Thread(target=run_flask)
thread.start()


In [ ]:
from pyngrok import ngrok

# Authenticate ngrok
ngrok.set_auth_token("YOUR_AUTH_TOKEN")

# Start the tunnel
ngrok_tunnel = ngrok.connect(5000)
print('Public URL:', ngrok_tunnel.public_url)
